# ============================================================
# SAT RESULTS DATA CLEANING & DATABASE INTEGRATION
# ============================================================

In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch

# ============================================================
## 1. Load Dataset
# ============================================================

In [2]:
# Load raw SAT results dataset
df = pd.read_csv('data/sat-results.csv')

# Preview the dataset
display(df.head())
# Inspect the dataset
print(df.info())
display(df.describe(include='all'))

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB
None


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
count,493,493,493,493,493,493,493,493.000000,388,376,402.000000
unique,478,478,175,164,177,163,164,NaN,3,3,NaN
top,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,s,s,s,s,s,NaN,x345,78%,NaN
freq,3,3,58,58,58,58,58,NaN,144,134,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562172.943205,NaN,NaN,2.564677
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,262138.627055,NaN,NaN,1.126443
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101855.000000,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,332013.000000,NaN,NaN,2.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,587220.000000,NaN,NaN,3.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,782993.000000,NaN,NaN,4.000000


# ============================================================
## 2. Remove Redundant / Invalid Columns
# ============================================================

In [3]:
# Drop duplicated SAT column with typographical error
df = df.drop(columns=['SAT Critical Readng Avg. Score'])

# ============================================================
## 3. Normalize Column Names
# ============================================================

In [4]:
# Standardize column names for consistency and SQL compatibility
df.columns = (
    df.columns
      .str.strip()                      # remove leading/trailing whitespace
      .str.lower()                      # convert to lowercase
      .str.replace(' ', '_', regex=True)  # replace spaces with underscores
      .str.replace(r'[^\w]', '', regex=True)  # remove special characters
)

print(df.columns)

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'internal_school_id', 'contact_extension',
       'pct_students_tested', 'academic_tier_rating'],
      dtype='object')


# ============================================================
## 4. Type Conversion and Parsing
# ============================================================

In [5]:
# Columns expected to be numeric
numeric_cols = [
    'num_of_sat_test_takers', 'sat_critical_reading_avg_score',
    'sat_math_avg_score', 'sat_writing_avg_score'
]
# Convert SAT-related columns to numeric, coercing invalid values to NaN
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Clean percentage column: strip '%' and convert to [0, 1] range
df['pct_students_tested'] = (
    df['pct_students_tested']
      .str.rstrip('%')
      .pipe(pd.to_numeric, errors='coerce') / 100
)

In [6]:
# Inspect distribution of academic tier ratings
df['academic_tier_rating'].value_counts()

academic_tier_rating
4.0    112
2.0    101
3.0     96
1.0     93
Name: count, dtype: int64

In [7]:
# Convert academic tier rating to nullable integer type
df['academic_tier_rating'] = (
    pd.to_numeric(df['academic_tier_rating'], errors='coerce')
      .astype('Int64')
)

# Inspect dataframe structure after conversions
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             493 non-null    object 
 1   school_name                     493 non-null    object 
 2   num_of_sat_test_takers          435 non-null    float64
 3   sat_critical_reading_avg_score  435 non-null    float64
 4   sat_math_avg_score              435 non-null    float64
 5   sat_writing_avg_score           435 non-null    float64
 6   internal_school_id              493 non-null    int64  
 7   contact_extension               388 non-null    object 
 8   pct_students_tested             376 non-null    float64
 9   academic_tier_rating            402 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 39.1+ KB


# ============================================================
## 5. Validation Checks
# ============================================================

In [8]:
# Identify invalid SAT scores outside the official 200–800 range
print(
    f'=== Invalid Math Scores ===\n'
    f'{df[(df["sat_math_avg_score"] < 200) | (df["sat_math_avg_score"] > 800)]["sat_math_avg_score"].value_counts()}\n'
)

print(
    f'=== Invalid Critical Reading Scores ===\n'
    f'{df[(df["sat_critical_reading_avg_score"] < 200) | (df["sat_critical_reading_avg_score"] > 800)]["sat_critical_reading_avg_score"].value_counts()}\n'
)

print(
    f'=== Invalid Writing Scores ===\n'
    f'{df[(df["sat_writing_avg_score"] < 200) | (df["sat_writing_avg_score"] > 800)]["sat_writing_avg_score"].value_counts()}\n'
)

# Validate percentage values
print(
    f'=== Valid pct_students_tested Values ===\n'
    f'{df[(df["pct_students_tested"] >= 0) & (df["pct_students_tested"] <= 1)]["pct_students_tested"].value_counts()}'
)

=== Invalid Math Scores ===
sat_math_avg_score
 999.0     2
 850.0     1
-10.0      1
 1100.0    1
Name: count, dtype: int64

=== Invalid Critical Reading Scores ===
Series([], Name: count, dtype: int64)

=== Invalid Writing Scores ===
Series([], Name: count, dtype: int64)

=== Valid pct_students_tested Values ===
pct_students_tested
0.78    134
0.85    125
0.92    117
Name: count, dtype: int64


# ============================================================
## 6. Filter Valid SAT Records
# ============================================================

In [9]:
# Retain only rows with valid SAT score ranges
for col in [
    'sat_critical_reading_avg_score',
    'sat_math_avg_score',
    'sat_writing_avg_score'
]:
    df_filtered = df[(df[col] >= 200) & (df[col] <= 800)]

# Reset index after filtering
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             435 non-null    object 
 1   school_name                     435 non-null    object 
 2   num_of_sat_test_takers          435 non-null    float64
 3   sat_critical_reading_avg_score  435 non-null    float64
 4   sat_math_avg_score              435 non-null    float64
 5   sat_writing_avg_score           435 non-null    float64
 6   internal_school_id              435 non-null    int64  
 7   contact_extension               344 non-null    object 
 8   pct_students_tested             329 non-null    float64
 9   academic_tier_rating            361 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 34.5+ KB


# ============================================================
## 7. Duplicate Detection and Removal
# ============================================================

In [10]:
# Identify fully duplicated rows
df_full_duplicates = df_filtered[df_filtered.duplicated(keep=False)]
df_full_duplicates = df_full_duplicates.reset_index(drop=True)

# Display sample duplicates for manual inspection
display(df_full_duplicates.head().sort_values(by='dbn'))

# Remove exact duplicates
df_filtered_unique = df_filtered.drop_duplicates().reset_index(drop=True)
df_filtered_unique.info()

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,02M419,LANDMARK HIGH SCHOOL,62.0,390.0,399.0,381.0,166135,x123,0.78,2
1,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264.0,407.0,440.0,393.0,892839,NaN,0.92,2
2,05M304,MOTT HALL HIGH SCHOOL,54.0,413.0,399.0,398.0,296405,x123,0.78,2
3,07X221,SOUTH BRONX PREPARATORY: A COLLEGE BOARD SCHOOL,65.0,364.0,378.0,348.0,277389,x345,0.92,<NA>
4,09X525,BRONX LEADERSHIP ACADEMY HIGH SCHOOL,114.0,384.0,394.0,388.0,906925,x234,0.92,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             421 non-null    object 
 1   school_name                     421 non-null    object 
 2   num_of_sat_test_takers          421 non-null    float64
 3   sat_critical_reading_avg_score  421 non-null    float64
 4   sat_math_avg_score              421 non-null    float64
 5   sat_writing_avg_score           421 non-null    float64
 6   internal_school_id              421 non-null    int64  
 7   contact_extension               334 non-null    object 
 8   pct_students_tested             317 non-null    float64
 9   academic_tier_rating            352 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 33.4+ KB


# ============================================================
## 8. Remove Non-Analytical / Non-Relational Columns
# ============================================================

In [11]:
# Drop columns not required for SAT performance analysis or DB integration
df_filtered_unique.drop(
    columns=[
        'school_name',
        'internal_school_id',
        'contact_extension',
        'pct_students_tested'
    ],
    inplace=True
)

df_filtered_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             421 non-null    object 
 1   num_of_sat_test_takers          421 non-null    float64
 2   sat_critical_reading_avg_score  421 non-null    float64
 3   sat_math_avg_score              421 non-null    float64
 4   sat_writing_avg_score           421 non-null    float64
 5   academic_tier_rating            352 non-null    Int64  
dtypes: Int64(1), float64(4), object(1)
memory usage: 20.3+ KB


# ============================================================
## 9. Convert pd.NA to Python None for psycopg2
# ============================================================

In [12]:
# Step 1: Convert pd.NA to Python None
df_cleaned = df_filtered_unique.copy()
df_cleaned = df_cleaned.replace({pd.NA: None})

# Step 2: Ensure Python-native types for psycopg2
for col in df_cleaned.columns:
    if pd.api.types.is_integer_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].astype(
            object)  # Int64 -> Python int / None
    elif pd.api.types.is_float_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].astype(float)

# Convert to list of tuples for batch insert
records = list(df_cleaned.itertuples(index=False, name=None))

# ============================================================
## 10. PostgreSQL Database Insertion
# ============================================================

In [13]:
try:
    # ============================================================
    # 1. Establish a secure connection to the PostgreSQL database
    # ============================================================
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",
        password="a9Am7Yy5r9_T7h4OF2GN",
        host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
        port="5432",
        sslmode="require"
    )
    cur = conn.cursor()  # Create a cursor object for executing SQL commands

    # ============================================================
    # 2. Create the SAT scores table if it does not already exist
    # ============================================================
    # Ensures the table structure is ready before inserting data
    cur.execute("""
    CREATE TABLE IF NOT EXISTS alexandra_dernova_sat_scores (
        dbn TEXT PRIMARY KEY,                           -- Unique school identifier
        num_of_sat_test_takers INTEGER,                -- Number of students who took SAT
        sat_critical_reading_avg_score FLOAT,          -- Average Critical Reading score
        sat_math_avg_score FLOAT,                       -- Average Math score
        sat_writing_avg_score FLOAT,                    -- Average Writing score
        academic_tier_rating INTEGER                    -- Tier rating (1–4)
    );
    """)

    # ============================================================
    # 3. Prepare parameterized INSERT query with conflict handling
    # ============================================================
    # ON CONFLICT ensures that if a record with the same dbn exists,
    # it will be updated instead of causing a duplicate key error
    insert_query = """
    INSERT INTO alexandra_dernova_sat_scores
    (
        dbn,
        num_of_sat_test_takers,
        sat_critical_reading_avg_score,
        sat_math_avg_score,
        sat_writing_avg_score,
        academic_tier_rating
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (dbn) DO UPDATE SET
        num_of_sat_test_takers = EXCLUDED.num_of_sat_test_takers,
        sat_critical_reading_avg_score = EXCLUDED.sat_critical_reading_avg_score,
        sat_math_avg_score = EXCLUDED.sat_math_avg_score,
        sat_writing_avg_score = EXCLUDED.sat_writing_avg_score,
        academic_tier_rating = EXCLUDED.academic_tier_rating;
    """

    # ============================================================
    # 4. Execute batch insert for performance
    # ============================================================
    # execute_batch is faster than inserting row by row and handles large datasets efficiently
    execute_batch(cur, insert_query, records, page_size=100)

    # Commit transaction to save changes
    conn.commit()
    print(f"Successfully inserted/updated {len(records)} records.")

except Exception as e:
    # Rollback transaction in case of any errors to maintain DB integrity
    conn.rollback()
    print("Error during database insertion:")
    print(e)

finally:
    # Close cursor and connection to release resources
    cur.close()
    conn.close()


Successfully inserted/updated 421 records.


# ============================================================
## 11. Final Verification & QA: Persist Cleaned Data and Validate PostgreSQL Load
# ============================================================

In [14]:
# Step 1: Save the cleaned SAT dataset to CSV for backup and offline inspection
df_filtered_unique.to_csv('data/sat-results-cleaned.csv', index=False)

# Step 2: Connect to the PostgreSQL database for verification
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()  # Cursor for executing SQL commands

# Step 3: Fetch all records from the SAT scores table to verify insertion
sql = """
SELECT *
FROM alexandra_dernova_sat_scores
"""
# Read SQL query results directly into a pandas DataFrame
df_sat_test = pd.read_sql(sql, conn)

# Step 4: Inspect the first few rows to ensure correctness
df_sat_test.head()

/var/folders/dj/qfwf7yns3z51j2jzcghy6r980000gn/T/ipykernel_20168/400252637.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sat_test = pd.read_sql(sql, conn)


,dbn,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,academic_tier_rating
0,01M292,29,355.0,404.0,363.0,2.0
1,01M448,91,383.0,423.0,366.0,3.0
2,01M450,70,377.0,402.0,370.0,3.0
3,01M458,7,414.0,401.0,359.0,4.0
4,01M509,44,390.0,433.0,384.0,2.0


In [15]:
# Close cursor and connection after verification
cur.close()
conn.close()